In [1]:
%pip install pyspark delta-spark pandas setuptools

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("aig") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

ss = configure_spark_with_delta_pip(builder).getOrCreate()

sql = ss.sql

25/02/09 16:08:39 WARN Utils: Your hostname, codespaces-f6f13d resolves to a loopback address: 127.0.0.1; using 10.0.2.145 instead (on interface eth0)
25/02/09 16:08:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13ae2ce8-9491-4ae2-a627-fdfd062adcc8;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 220ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

In [5]:
import subprocess

files = [
    'name.basics.tsv.gz',
    'title.akas.tsv.gz',
    'title.basics.tsv.gz',
    'title.crew.tsv.gz',
    'title.episode.tsv.gz',
    'title.principals.tsv.gz',
    'title.ratings.tsv.gz'
]

for file in files:
    subprocess.run(["wget", "https://datasets.imdbws.com/" + file, "-nc", "-O", "/tmp/" + file])    

for file in files:
    table_name = file.replace('.tsv.gz', '').replace('.', '_')
    ss.read.csv("/tmp/" + file, header=True, sep='\t', nullValue='\\N').createOrReplaceTempView(table_name + "_csv")

File ‘/tmp/name.basics.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.akas.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.basics.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.crew.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.episode.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.principals.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.ratings.tsv.gz’ already there; not retrieving.


In [ ]:
from pathlib import Path

def load_spark_tables(ss: pyspark.sql.SparkSession, schema: str, warehouse_path: Path):
    ss.sql("CREATE SCHEMA IF NOT EXISTS " + schema).collect()

    if warehouse_path.exists():
        for item in warehouse_path.glob('*'):
            table_name = schema + "." + item.name
            print("LOAD TABLE " + table_name)
            ss.catalog.createTable(table_name, path=item.absolute().as_posix(), source='delta')

load_spark_tables(ss, "baseline", Path("./tests") / 'baseline-warehouse' / 'baseline.db')


In [17]:
sql("""
WITH report AS
(
SELECT 
    explode(split(genres, ',')) AS genre, 
    CAST(numVotes AS INT) as numVotes, 
    CAST(averageRating AS DOUBLE) AS averageRating
FROM title_basics_csv
JOIN title_ratings_csv USING (tconst)
)

SELECT 
    genre, 
    SUM(numVotes) AS numVotes, 
    AVG(averageRating) AS averageRating
FROM report 
GROUP BY genre

""").show(truncate=False)

+-----------+-------------+------------------+
|genre      |sum(numVotes)|avg(averageRating)|
+-----------+-------------+------------------+
|Crime      |314477334    |7.111348385399751 |
|Romance    |177515619    |6.963582672300842 |
|Thriller   |231993717    |6.437873087534484 |
|Adventure  |410107979    |7.1079752857222305|
|Drama      |829684185    |7.054152508418149 |
|War        |29688491     |7.04596967278532  |
|Documentary|31145139     |7.237939083924439 |
|Reality-TV |5359464      |7.059994501619527 |
|Family     |68794179     |7.085333261150007 |
|Fantasy    |143976380    |7.072739683037801 |
|Game-Show  |2714331      |7.097522846288134 |
|Adult      |911819       |6.2423683759218385|
|History    |44615006     |7.355752819589149 |
|Mystery    |173497868    |7.0804702428757045|
|Musical    |12162630     |6.612117537978759 |
|Animation  |136274659    |7.1375507431547165|
|Music      |29593415     |7.072352091748102 |
|Film-Noir  |4029261      |6.455414746543779 |
|Short      |